Gaussian Mixture


In [290]:
import pandas as pd
import numpy as np

In [291]:
df = pd.read_csv('combined_data.csv')
df

,current_1,current_2,current_3
0,2040,2361,38
1,2031,2365,52
2,2031,2370,53
3,2025,2372,54
4,2023,2379,54
...,...,...,...
3169995,2214,1978,368
3169996,2217,1985,358
3169997,2218,1989,346
3169998,2220,1996,337


manually

In [292]:
# grouped = df.groupby(df.index // 10000)
# means = grouped.mean()
# variances = grouped.var()
# stds = grouped.std()
# cols = pd.MultiIndex.from_product([df.columns, ['mean', 'var', 'std','harmonic_magnitude_1','harmonic_magnitude_2','harmonic_magnitude_3']])
# result = pd.DataFrame(columns=cols)

# # # Add new column for group index
# # df['group'] = df.index // 10000

# # Group by group index and calculate mean, variance, and standard deviation
# grouped = df.groupby(df.index // 10000)
# means = grouped.mean()
# variances = grouped.var()
# stds = grouped.std()

# # Create new DataFrame with MultiIndex columns
# cols = pd.MultiIndex.from_product([df.columns, ['mean', 'var', 'std']])
# result = pd.DataFrame(columns=cols)

# # Populate new DataFrame with means, variances, and standard deviations
# for col in df.columns:
#     result[col, 'mean'] = means[col]
#     result[col, 'var'] = variances[col]
#     result[col, 'std'] = stds[col]

# # Reorder columns
# # result = result.reindex(sorted(result.columns), axis=1)

# # Print result
# # print(result.columns)


In [293]:
def _get_current_unbalance( fft_data: np.ndarray) -> float:
        """
        Calculates the current unbalance from the FFT of the current waveform data.

        Parameters:
            fft_data (numpy.ndarray): The FFT of the current waveform data.

        Returns:
            float: The current unbalance.
        """
        phase_angles = np.angle(fft_data, deg=True)
        phase_angles_diff = np.diff(phase_angles, axis=0)
        current_unbalance = np.max(np.abs(phase_angles_diff))
        return current_unbalance

def _get_rms_current(df: np.ndarray) -> float:
    """
    Calculates the RMS current from the current waveform data.

    Parameters:
        df (pandas.DataFrame): The current waveform data as a pandas DataFrame.

    Returns:
        float: The RMS current.
    """
    rms_current = np.sqrt(np.mean(df**2))
    return rms_current

def _get_harmonic_magnitudes( fft_data: np.ndarray, fs=10000, ffreq=50):
    """
    Calculates the magnitudes of the harmonic components from the FFT of the current waveform data.

    Parameters:
        fft_data (numpy.ndarray): The FFT of the current waveform data.
        fs (float): The sampling frequency of the current waveform data.
        ffreq (float): The fundamental frequency of the current waveform.

    Returns:
        numpy.ndarray: The magnitudes of the harmonic components.
    """
    fundamental_freq = ffreq  # or 60 Hz depending on the AC power supply frequency
    harmonic_freqs = [2*fundamental_freq, 3*fundamental_freq,
                        4*fundamental_freq]  # extract 2nd, 3rd and 4th harmonics
    # compute indices of harmonic components
    harmonic_indices = [int(freq/fs * (len(fft_data)/2))
                        for freq in harmonic_freqs]

    # extract magnitudes of harmonic components
    harmonic_magnitudes = np.abs(fft_data[harmonic_indices])

    # normalize harmonic magnitudes using z-score normalization
    # mean = np.mean(harmonic_magnitudes)
    # std = np.std(harmonic_magnitudes)
    # harmonic_magnitudes_normalized = (harmonic_magnitudes - mean) / std

    return harmonic_magnitudes

def _get_thd(fft_data: np.ndarray, sampling_rate: int, fundamental_frequency: int) -> float:
    """
    Calculates the total harmonic distortion (THD) of the current data.

    Parameters:
        fft_data (numpy.ndarray): The FFT of the current data.
        sampling_rate (int): The sampling rate of the current data 
        fundamental_frequency (int): The fundamental frequency of the current waveform.

    Returns:
    --------
    thd: float
        The THD value.
    """
    fundamental_index = int(
        fundamental_frequency * (len(fft_data)/2) / sampling_rate)
    thd = np.sqrt(np.sum(np.abs(fft_data[fundamental_index-2:fundamental_index+3])**2) - np.abs(
        fft_data[fundamental_index])**2) / np.abs(fft_data[fundamental_index])
    return thd


In [294]:
def _get_features(df):

    features = df.groupby(df.index // 10000).agg(['mean', 'var', 'std'])
    mags = []
    thds = []
    rmss = []
    unbals = []
    fft_data = np.fft.fft(df)
    for i in range(0,df.shape[0],10000):
        mags.append(_get_harmonic_magnitudes(fft_data[i:i+10000]))
        thds.append(_get_thd(fft_data[i:i+10000], 10000, 50))
        rmss.append(_get_rms_current(df[i:i+10000]))
        unbals.append(_get_current_unbalance(fft_data[i:i+10000]))

        
    for i, row in features.iterrows():
        harmonic_magnitudes = mags[i]
        thd = thds[i]
        rms = rmss[i]
        unbal = unbals[i]
        features.loc[i, ('current_1', 'harmonic_2')] = harmonic_magnitudes[0][0]
        features.loc[i, ('current_1', 'harmonic_3')] = harmonic_magnitudes[0][1]
        features.loc[i, ('current_1', 'harmonic_4')] = harmonic_magnitudes[0][2]
        features.loc[i, ('current_2', 'harmonic_2')] = harmonic_magnitudes[1][0]
        features.loc[i, ('current_2', 'harmonic_3')] = harmonic_magnitudes[1][1]
        features.loc[i, ('current_2', 'harmonic_4')] = harmonic_magnitudes[1][2]
        features.loc[i, ('current_3', 'harmonic_2')] = harmonic_magnitudes[2][0]
        features.loc[i, ('current_3', 'harmonic_3')] = harmonic_magnitudes[2][1]
        features.loc[i, ('current_3', 'harmonic_4')] = harmonic_magnitudes[2][2]

        features.loc[i, ('current_1', 'thd')] = thd[0]
        features.loc[i, ('current_2', 'thd')] = thd[1]
        features.loc[i, ('current_3', 'thd')] = thd[2]

        features.loc[i, ('current_1', 'rms')] = rms[0]
        features.loc[i, ('current_2', 'rms')] = rms[1]
        features.loc[i, ('current_3', 'rms')] = rms[2]
        features.loc[i, ('current_unbalance', '')] = unbal

    features.columns = features.columns.to_flat_index()
    features.columns = ['_'.join(column) for column in features.columns]
    return [features, fft_data]




In [295]:
features,fft = _get_features(df)

c:\Users\deepdesk\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [296]:
features.head()

,current_1_mean,current_1_var,current_1_std,current_2_mean,current_2_var,current_2_std,current_3_mean,current_3_var,current_3_std,current_1_harmonic_2,...,current_3_harmonic_2,current_3_harmonic_3,current_3_harmonic_4,current_1_thd,current_2_thd,current_3_thd,current_1_rms,current_2_rms,current_3_rms,current_unbalance_
0,1987.6003,32783.557096,181.062302,2174.0711,33020.711316,181.716018,440.3539,75256.979553,274.330056,4729.0,...,4593.0,1735.474287,1735.474287,2.313372,6.785160,6.785160,1995.829459,2181.651337,518.807297,18.686054
1,1982.4803,33379.063118,182.699379,2165.3532,32815.538604,181.150596,435.3327,73915.736984,271.874488,4601.0,...,4497.0,1978.679105,1978.679105,2.206607,8.168106,8.168106,1990.880173,2172.916643,513.247411,18.875276
2,1980.2381,33015.768385,181.702417,2170.6435,32957.610569,181.542311,428.8870,74261.647196,272.509903,4543.0,...,4629.0,1518.676727,1518.676727,2.495040,5.604163,5.604163,1988.556109,2178.221182,508.132148,27.656414
3,1978.7542,33224.950477,182.277125,2170.5052,32863.028676,181.281628,428.6128,74053.162992,272.127108,4428.0,...,4692.0,1274.577577,1274.577577,2.406731,6.406529,6.406529,1987.131051,2178.061653,507.695470,20.568046
4,1982.3731,33032.491746,181.748430,2164.9059,33034.596305,181.754219,432.9330,74385.515663,272.737082,4556.0,...,4590.0,1681.609943,1681.609943,2.258590,7.369790,7.369790,1990.686388,2172.521311,511.672805,20.326629


In [297]:
from sklearn.mixture import GaussianMixture

In [298]:
gmm = GaussianMixture(n_components=1, covariance_type='full')

In [299]:
gmm.fit(features)

c:\Users\deepdesk\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


GaussianMixture()

In [300]:
features.sample(frac=0.5)

,current_1_mean,current_1_var,current_1_std,current_2_mean,current_2_var,current_2_std,current_3_mean,current_3_var,current_3_std,current_1_harmonic_2,...,current_3_harmonic_2,current_3_harmonic_3,current_3_harmonic_4,current_1_thd,current_2_thd,current_3_thd,current_1_rms,current_2_rms,current_3_rms,current_unbalance_
167,1987.3133,32777.526096,181.045646,2173.7994,32470.887248,180.196801,444.2577,73880.352126,271.809404,4530.0,...,4494.0,2020.195040,2020.195040,2.288820,7.039880,7.039880,1995.542132,2181.254564,520.805019,20.850624
131,1984.9608,33643.882452,183.422688,2169.0872,31756.754872,178.204250,434.9028,73992.832035,272.016235,4454.0,...,4651.0,1490.692457,1490.692457,2.456816,5.645173,5.645173,1993.416639,2176.394463,512.957969,23.581685
57,1981.7891,32757.332554,180.989869,2166.5486,32874.236862,181.312539,429.7468,74890.771567,273.661783,4454.0,...,4459.0,2102.434541,2102.434541,2.356132,6.428128,6.428128,1990.035702,2174.121382,509.475804,18.785411
42,1977.3821,33132.885388,182.024409,2163.6041,32902.492212,181.390441,422.5831,74084.925787,272.185462,4683.0,...,4442.0,2090.540361,2090.540361,2.206056,8.099785,8.099785,1985.741560,2171.193659,502.646987,19.947207
76,1980.8027,33283.875560,182.438690,2169.4545,32061.013831,179.055896,429.2384,74646.297195,273.214746,4704.0,...,4455.0,2093.304326,2093.304326,2.230934,7.696154,7.696154,1989.185734,2176.830410,508.806876,21.044043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,2017.6248,32629.609986,180.636680,2196.8195,32626.726392,180.628698,494.6534,74554.085477,273.045940,4837.0,...,4567.0,2084.124516,2084.124516,2.168962,9.015843,9.015843,2025.694000,2204.232152,565.003200,22.252779
263,1985.5395,33024.432883,181.726258,2167.7296,32687.634847,180.797220,432.6673,75280.493660,274.372910,4452.0,...,4714.0,1191.613612,1191.613612,2.483244,5.668111,5.668111,1993.837565,2175.255384,512.322123,24.387115
151,1986.2995,33244.844284,182.331688,2173.6952,32344.889186,179.846849,449.2048,73697.669224,271.473146,4760.0,...,4469.0,2154.894197,2154.894197,2.161422,9.066612,9.066612,1994.649649,2181.121839,524.857363,27.930349
230,2015.5736,32472.498233,180.201271,2198.6288,32220.627273,179.501051,493.8677,75457.086305,274.694533,4843.0,...,4581.0,2060.417676,2060.417676,2.224650,7.835070,7.835070,2023.612163,2205.943337,565.114896,24.276196


In [301]:
eval_df = features.sample(frac=0.5)

In [302]:
scores = gmm.score_samples(features)

In [303]:
scores.max(),scores.min(),scores.mean()

(-20.62308262768359, -176.03354207552104, -27.406391846429173)

In [304]:
mean_score = np.mean(scores)
std_score = np.std(scores)

In [305]:
k = 3
threshold = mean_score - k * std_score

In [306]:
threshold

-59.40404985032065

In [307]:

anomaly_threshold = gmm.score_samples(eval_df) < threshold


In [308]:
import copy
eval_df_1 = copy.deepcopy(eval_df)

Self evaluating

In [309]:
eval_df_1['anomaly'] = anomaly_threshold.astype(int)

In [310]:
len(eval_df_1.loc[eval_df_1['anomaly'] == 1])

1

In [311]:
eval_df_1.loc[eval_df_1['anomaly'] == 1]

,current_1_mean,current_1_var,current_1_std,current_2_mean,current_2_var,current_2_std,current_3_mean,current_3_var,current_3_std,current_1_harmonic_2,...,current_3_harmonic_3,current_3_harmonic_4,current_1_thd,current_2_thd,current_3_thd,current_1_rms,current_2_rms,current_3_rms,current_unbalance_,anomaly
242,1996.6585,5704.959574,75.531183,2182.6425,5684.584952,75.396187,458.7966,13228.947723,115.017163,4761.0,...,1244.678673,1244.678673,2.26131,7.349606,7.349606,1998.086473,2183.944207,472.992542,17.385438,1


possible real time function

In [312]:
# while True:
#     current_readings = get_current_readings()
#     current_features = pd.DataFrame([current_readings]).agg(['mean', 'var', 'std'])
#     is_anomaly = gmm.score_samples(current_features) < -30
#     if is_anomaly:
#         alert_operator()

Adding Anomalies 

In [313]:
def add_anomalies(dataframe, number_of_anomalies=0, data_length=0):

        # Determine the upper bound of the data to which anomalies should be added.
        upper_bound = number_of_anomalies if number_of_anomalies else len(dataframe)
        
        # Determine the length of the data to which anomalies should be added.
        length = data_length if data_length else len(dataframe)

        # Make a deep copy of the data up to the upper bound and length.
        df = copy.deepcopy(dataframe[0:length])
        noise_df = df.iloc[:upper_bound,:]

        # Calculate the noise amplitude based on the signal amplitude.
        noise_amplitude = 0.1 * np.max(np.abs(df))

        # Add noise to the signal using amplitude scaling.
        noise = np.random.normal(0, noise_amplitude, noise_df.shape)
        
        df.iloc[:upper_bound,:] += noise

        return df


In [314]:
anomalous_data = add_anomalies(df,5000,10000)

c:\Users\deepdesk\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [315]:
anomalous_features,anomalous_fft = _get_features(anomalous_data)


c:\Users\deepdesk\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [316]:
anomalous_features

,current_1_mean,current_1_var,current_1_std,current_2_mean,current_2_var,current_2_std,current_3_mean,current_3_var,current_3_std,current_1_harmonic_2,...,current_3_harmonic_2,current_3_harmonic_3,current_3_harmonic_4,current_1_thd,current_2_thd,current_3_thd,current_1_rms,current_2_rms,current_3_rms,current_unbalance_
0,1983.973584,58212.195565,241.272036,2173.261625,63059.637334,251.11678,439.585627,78868.973205,280.836204,5502.460745,...,4247.119789,1658.41784,1658.41784,2.196913,6.481964,6.481964,1998.588891,2187.720143,521.628805,60.34165


In [317]:
is_anomaly = gmm.score_samples(anomalous_features) < threshold
if is_anomaly:
    print('anomalous')

anomalous


In [318]:
is_anomaly

array([ True])

In [324]:
features

,current_1_mean,current_1_var,current_1_std,current_2_mean,current_2_var,current_2_std,current_3_mean,current_3_var,current_3_std,current_1_harmonic_2,...,current_3_harmonic_2,current_3_harmonic_3,current_3_harmonic_4,current_1_thd,current_2_thd,current_3_thd,current_1_rms,current_2_rms,current_3_rms,current_unbalance_
0,1987.6003,32783.557096,181.062302,2174.0711,33020.711316,181.716018,440.3539,75256.979553,274.330056,4729.0,...,4593.0,1735.474287,1735.474287,2.313372,6.785160,6.785160,1995.829459,2181.651337,518.807297,18.686054
1,1982.4803,33379.063118,182.699379,2165.3532,32815.538604,181.150596,435.3327,73915.736984,271.874488,4601.0,...,4497.0,1978.679105,1978.679105,2.206607,8.168106,8.168106,1990.880173,2172.916643,513.247411,18.875276
2,1980.2381,33015.768385,181.702417,2170.6435,32957.610569,181.542311,428.8870,74261.647196,272.509903,4543.0,...,4629.0,1518.676727,1518.676727,2.495040,5.604163,5.604163,1988.556109,2178.221182,508.132148,27.656414
3,1978.7542,33224.950477,182.277125,2170.5052,32863.028676,181.281628,428.6128,74053.162992,272.127108,4428.0,...,4692.0,1274.577577,1274.577577,2.406731,6.406529,6.406529,1987.131051,2178.061653,507.695470,20.568046
4,1982.3731,33032.491746,181.748430,2164.9059,33034.596305,181.754219,432.9330,74385.515663,272.737082,4556.0,...,4590.0,1681.609943,1681.609943,2.258590,7.369790,7.369790,1990.686388,2172.521311,511.672805,20.326629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,1981.0900,33103.129613,181.942655,2169.3031,32608.981529,180.579571,436.4242,75003.547809,273.867756,4468.0,...,4574.0,1752.847683,1752.847683,2.219257,7.937776,7.937776,1989.426402,2176.805380,515.230172,20.261872
313,1984.3788,33107.962507,181.955936,2165.6269,32697.299226,180.823945,429.8828,74541.981862,273.023775,4455.0,...,4598.0,1629.288495,1629.288495,2.258475,7.360745,7.360745,1992.702656,2173.162189,509.248220,27.362831
314,1981.1716,33125.368090,182.003758,2166.5496,32823.523692,181.172635,430.5510,74550.616861,273.039588,4463.0,...,4573.0,1735.114117,1735.114117,2.229874,7.786725,7.786725,1989.513248,2174.110717,509.820876,26.060691
315,1983.0589,33408.837915,182.780847,2165.2676,32843.622352,181.228095,429.9598,73998.240808,272.026177,4487.0,...,4675.0,1332.334793,1332.334793,2.528244,5.462741,5.462741,1991.463808,2172.837803,508.779196,25.601103


In [323]:
features.columns

Index(['current_1_mean', 'current_1_var', 'current_1_std', 'current_2_mean',
       'current_2_var', 'current_2_std', 'current_3_mean', 'current_3_var',
       'current_3_std', 'current_1_harmonic_2', 'current_1_harmonic_3',
       'current_1_harmonic_4', 'current_2_harmonic_2', 'current_2_harmonic_3',
       'current_2_harmonic_4', 'current_3_harmonic_2', 'current_3_harmonic_3',
       'current_3_harmonic_4', 'current_1_thd', 'current_2_thd',
       'current_3_thd', 'current_1_rms', 'current_2_rms', 'current_3_rms',
       'current_unbalance_'],
      dtype='object')

In [325]:
import sys
sys.path.append('../induction_motor_anomaly_detection/')

In [326]:
import modules,scaler

In [ ]:
#TODO use these libraries here for the demo